# Lab-1.7: SFT 基準模型

**目標**: 訓練 SFT 基準, 作為 DPO 的參考模型

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer

print('SFT 訓練...')
model = AutoModelForCausalLM.from_pretrained('gpt2')
print(f'模型參數: {sum(p.numel() for p in model.parameters())/1e6:.1f}M')